# Udacity Data Engineering Capstone

# US Immigration DW

## Overview
The purpose of this data engineering capstone project is to give apply data engineering concepts and tools such as ETL pipelines using Spark, AWS S3, Amazon Redshift, etc.

## Scenario
We have been contracted by the U.S. Customs and Border Protection to help them with the flood of immigration data. We aim to model and create a brand new analytics solution on top of the state-of-the-art technolgies available to enable them to unleash insights from data then providing better customer experiences when coming to the US.

## Structure of the Project
Following the Udacity guide for this project, we structured this documentation with steps below:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

## Step 1: Scope the Project and Gather Data

### The Scope 
The main deliverable of our work here will be to create a data warehouse in the cloud that will support the officials in answering questions through analytics tables and dashboards. Additionally, as we develop a general source-of-truth database, the immigration officials could open the solution through a web API so backend web services could query the warehouse for information relating to international visitors.

### The Data
For this work we have used the immigration, the global temperature and demographics datasets as well as the descriptions contained in the `I94_SAS_Labels_Descriptions.SAS` file.

### The Architecture
The whole solution is cloud based on top of __Amazon Web Services (AWS)__. First, all the datasets were preprocessed with __Apache Spark__ and stored in a staging area in __AWS S3__ bucket. Then, we loaded those to a __Amazon Redshift__ cluster using an __Apache Airflow__ pipeline that transfer and check the quality of the data to finally provide our customers a data mart for their convenient analysis.


The main information and questions a user may want to extract from the data mart would be:

* Visitors by nationality.
* Visitors by origin.
* Visitors by airline.
* Correlations between destination in the U.S and the source country.
* Correlations between destination in the U.S and source climates.
* Correlations between immigration by source region, and the source region temperature.
* Correlations between visitor demographics, and states visited.


## Step 2: Explore and Assess the Data

_To familiarize ourselves with the data provided by Udacity we have done an exhaustive exploratory data analysis ([EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis)) checking what data would be useful and what preprocessing steps we should take in order to clean, organize and join the various datasets in a meaningful data model._

In the following sections we briefly describe the datasets provided and give a summarized idea on the reasons we took into consideration when deciding what data to use.

__Immigration Data__

For decades, U.S. immigration officers issued the I-94 Form (Arrival/Departure Record) to foreign visitors (e.g., business visitors, tourists and foreign students) who lawfully entered the United States. The I-94 was a small white paper form that a foreign visitor received from cabin crews on arrival flights and from U.S. Customs and Border Protection at the time of entry into the United States. It listed the traveler's immigration category, port of entry, data of entry into the United States, status expiration date and had a unique 11-digit identifying number assigned to it. Its purpose was to record the traveler's lawful admission to the United States.

#### Imports

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, rand
from pyspark.sql.functions import isnan, when, count, col
import configparser
import psycopg2

#### I94 Immigration Data pull

In [2]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [3]:
imm_data = spark.read.parquet("data/sas_data")
print(imm_data.count())
imm_data.limit(10).toPandas()

3096313


cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode  \
0  5748517.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
1  5748518.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
2  5748519.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
3  5748520.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
4  5748521.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
5  5748522.0  2016.0     4.0   245.0   464.0     HHW  20574.0      1.0   
6  5748523.0  2016.0     4.0   245.0   464.0     HHW  20574.0      1.0   
7  5748524.0  2016.0     4.0   245.0   464.0     HHW  20574.0      1.0   
8  5748525.0  2016.0     4.0   245.0   464.0     HOU  20574.0      1.0   
9  5748526.0  2016.0     4.0   245.0   464.0     LOS  20574.0      1.0   

  i94addr  depdate  ...  entdepu  matflag  biryear   dtaddto gender insnum  \
0      CA  20582.0  ...     None        M   1976.0  10292016      F   None   
1      NV  20591.0  ...     None        M   1984.0  10292016      F   None   
2      WA  20582.0  ...     None        M   1987.0  10292016      M   None   
3      WA  20588.0  ...     None        M   1987.0  10292016      F   None   
4      WA  20588.0  ...     None        M   1988.0  10292016      M   None   
5      HI  20579.0  ...     None        M   1959.0  10292016      M   None   
6      HI  20586.0  ...     None        M   1950.0  10292016      F   None   
7      HI  20586.0  ...     None        M   1975.0  10292016      F   None   
8      FL  20581.0  ...     None        M   1989.0  10292016      M   None   
9      CA  20581.0  ...     None        M   1990.0  10292016      F   None   

  airline        admnum  fltno visatype  
0      QF  9.495387e+10  00011       B1  
1      VA  9.495562e+10  00007       B1  
2      DL  9.495641e+10  00040       B1  
3      DL  9.495645e+10  00040       B1  
4      DL  9.495639e+10  00040       B1  
5      NZ  9.498180e+10  00010       B2  
6      NZ  9.497969e+10  00010       B2  
7      NZ  9.497975e+10  00010       B2  
8      NZ  9.497325e+10  00028       B2  
9      NZ  9.501355e+10  00002       B2  

[10 rows x 28 columns]

__Data Dictionary__: Here, we describe the various fields of the dataset. Some descriptions were not clear enough so we had to make assumptions about the meaning.

| Column Name | Description |
| :--- | :--- |
| CICID* | ID that uniquely identify one record in the dataset |
| I94YR | 4 digit year |
| I94MON | Numeric month |
| I94CIT | 3 digit code of source city for immigration (Born country) |
| I94RES | 3 digit code of source country for immigration (Residence country) |
| I94PORT | Port addmitted through |
| ARRDATE | Arrival date in the USA |
| I94MODE | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) |
| I94ADDR | State of arrival |
| DEPDATE | Departure date |
| I94BIR | Age of Respondent in Years |
| I94VISA | Visa codes collapsed into three categories: (1 = Business; 2 = Pleasure; 3 = Student) |
| COUNT | Used for summary statistics |
| DTADFILE | Character Date Field |
| VISAPOST | Department of State where where Visa was issued |
| OCCUP | Occupation that will be performed in U.S. |
| ENTDEPA | Arrival Flag. Whether admitted or paroled into the US |
| ENTDEPD | Departure Flag. Whether departed, lost visa, or deceased |
| ENTDEPU | Update Flag. Update of visa, either apprehended, overstayed, or updated to PR |
| MATFLAG | Match flag |
| BIRYEAR | 4 digit year of birth |
| DTADDTO | Character date field to when admitted in the US |
| GENDER | Gender |
| INSNUM | INS number |
| AIRLINE | Airline used to arrive in U.S. |
| ADMNUM | Admission number, should be unique and not nullable |
| FLTNO | Flight number of Airline used to arrive in U.S. |
| VISATYPE | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |

The immigration dataset is our fact so that will be at the center of the star schema model of our data warehouse.

#### U.S. City Demographic Data pull

In [4]:
city_dem_data = pd.read_csv('data/us-cities-demographics.csv', sep=';')
print(city_dem_data.info())
city_dem_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   City                    2891 non-null   object 
 1   State                   2891 non-null   object 
 2   Median Age              2891 non-null   float64
 3   Male Population         2888 non-null   float64
 4   Female Population       2888 non-null   float64
 5   Total Population        2891 non-null   int64  
 6   Number of Veterans      2878 non-null   float64
 7   Foreign-born            2878 non-null   float64
 8   Average Household Size  2875 non-null   float64
 9   State Code              2891 non-null   object 
 10  Race                    2891 non-null   object 
 11  Count                   2891 non-null   int64  
dtypes: float64(6), int64(2), object(4)
memory usage: 271.2+ KB
None


City          State  Median Age  Male Population  \
0     Silver Spring       Maryland        33.8          40601.0   
1            Quincy  Massachusetts        41.0          44129.0   
2            Hoover        Alabama        38.5          38040.0   
3  Rancho Cucamonga     California        34.5          88127.0   
4            Newark     New Jersey        34.6         138040.0   

   Female Population  Total Population  Number of Veterans  Foreign-born  \
0            41862.0             82463              1562.0       30908.0   
1            49500.0             93629              4147.0       32935.0   
2            46799.0             84839              4819.0        8229.0   
3            87105.0            175232              5821.0       33878.0   
4           143873.0            281913              5829.0       86253.0   

   Average Household Size State Code                       Race  Count  
0                    2.60         MD         Hispanic or Latino  25924  
1                    2.39         MA                      White  58723  
2                    2.58         AL                      Asian   4759  
3                    3.18         CA  Black or African-American  24437  
4                    2.73         NJ                      White  76402

__Data Dictionary__

| Column Name | Description |
| :--- | :--- |
| City | Name of the city |
| State | US state of the city |
| Median Age | The median of the age of the population |
| Male Population | Number of the male population |
| Female Population | Number of the female population |
| Total Population | Number of the total population |
| Number of Veterans | Number of veterans living in the city |
| Foreign-born | Number of residents of the city that were not born in the city |
| Average Household Size | Average size of the houses in the city |
| State Code | Code of the state of the city |
| Race | Race class |
| Count | Number of individual of each race |

The `US Cities Demographics` is the source of the STATE dimension in our data model. We aggregated the dataset by State and pivoted the `Race` and `Count` columns in order to make each different value of Race to be a column. That way we create a complete table of statistics that summarizes the information for every US state.

#### Airport Data

The airport codes may refer to either [IATA](https://en.wikipedia.org/wiki/IATA_airport_code) airport code, a three-letter code which is used in passenger reservation, ticketing and baggage-handling systems, or the [ICAO](https://en.wikipedia.org/wiki/ICAO_airport_code) airport code which is a four letter code used by ATC systems and for airports that do not have an IATA airport code (from wikipedia).

Airport codes from around the world. Downloaded from public domain source http://ourairports.com/data/ who compiled this data from multiple different sources.

`airport-codes.csv` contains the list of all airport codes, the attributes are identified in datapackage description. Some of the columns contain attributes identifying airport locations, other codes (IATA, local if exist) that are relevant to identification of an airport.
Original source url is http://ourairports.com/data/airports.csv (stored in archive/data.csv).

In [5]:
airport_code_data = pd.read_csv('data/airport-codes_csv.csv')
print(airport_code_data.info())
airport_code_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ident         55075 non-null  object 
 1   type          55075 non-null  object 
 2   name          55075 non-null  object 
 3   elevation_ft  48069 non-null  float64
 4   continent     27356 non-null  object 
 5   iso_country   54828 non-null  object 
 6   iso_region    55075 non-null  object 
 7   municipality  49399 non-null  object 
 8   gps_code      41030 non-null  object 
 9   iata_code     9189 non-null   object 
 10  local_code    28686 non-null  object 
 11  coordinates   55075 non-null  object 
dtypes: float64(1), object(11)
memory usage: 5.0+ MB
None


ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

#### Global Temperature Data

There are a range of organizations that collate climate trends data. The three most cited land and ocean temperature data sets are NOAA’s MLOST, NASA’s GISTEMP and the UK’s HadCrut.

The Berkeley Earth, which is affiliated with Lawrence Berkeley National Laboratory, has repackaged the data from a newer compilation put it all together. The Berkeley Earth Surface Temperature Study combines 1.6 billion temperature reports from 16 pre-existing archives. It is nicely packaged and allows for slicing into interesting subsets (for example by country). They publish the source data and the code for the transformations they applied. They also use methods that allow weather observations from shorter time series to be included, meaning fewer observations need to be thrown away.

In the original dataset from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data), several files are available but in this capstone project we will be using only the `GlobalLandTemperaturesByCity`.

In [6]:
temp_data = pd.read_csv('data/GlobalLandTemperaturesByCity.csv')
print(temp_data.info())
temp_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   dt                             object 
 1   AverageTemperature             float64
 2   AverageTemperatureUncertainty  float64
 3   City                           object 
 4   Country                        object 
 5   Latitude                       object 
 6   Longitude                      object 
dtypes: float64(2), object(5)
memory usage: 459.2+ MB
None


dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E

__Data Dictionary__

| Column Name | Description |
| :--- | :--- |
| dt | Date in format YYYY-MM-DD |
| AverageTemperature | Average temperature of the city in a given date |
| City | City Name |
| Country | Country Name |
| Latitude | Latitude |
| Longitude | Longitude |

The dataset provides a long period of the world's temperature (from year 1743 to 2013). However, since the immigration dataset only has data of the US National Tourism Office in the year of 2016, the vast majority of the data here seems not to be suitable. We then decided to aggregate this dataset by country, averaging the temperatures and use this reduced table to join with `lookup\I94CIT_I94RES.csv` lookup table (extracted from `I94_SAS_Labels_Descriptions.SAS`) resulting in the COUNTRY dimension of our model.

> If we had temperatures of the year 2016 we could have provided an interesting analysis crossing the two tables (immigration and temperatures) in order to see how the waves of immigration to the US relate to the changes in the temperature. But this is just unfeasible due to the different dates.

In [7]:
world_temperature = temp_data.groupby(["Country"]).agg({"AverageTemperature": "mean", 
                                                                        "Latitude": "first", "Longitude": "first"}).reset_index()

world_temperature.head()

Country  AverageTemperature Latitude Longitude
0  Afghanistan           13.816497   36.17N    69.61E
1      Albania           15.525828   40.99N    19.17E
2      Algeria           17.763206   36.17N     3.98E
3       Angola           21.759716   12.05S    13.15E
4    Argentina           16.999216   39.38S    62.43W

## Data Exploration & Modeling

### Data Prep

In [8]:
def sas_program_file_value_parser(sas_source_file, value, columns):
    """Parses SAS Program file to return value as pandas dataframe
    Args:
        sas_source_file (str): SAS source code file.
        value (str): sas value to extract.
        columns (list): list of 2 containing column names.
    Return:
        None
    """
    file_string = ''
    
    with open(sas_source_file) as f:
        file_string = f.read()
    
    file_string = file_string[file_string.index(value):]
    file_string = file_string[:file_string.index(';')]
    
    line_list = file_string.split('\n')[1:]
    codes = []
    values = []
    
    for line in line_list:
        
        if '=' in line:
            code, val = line.split('=')
            code = code.strip()
            val = val.strip()

            if code[0] == "'":
                code = code[1:-1]

            if val[0] == "'":
                val = val[1:-1]

            codes.append(code)
            values.append(val)
        
            
    return pd.DataFrame(zip(codes,values), columns=columns)

In [9]:
i94cit_res = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94cntyl', ['code', 'country'])
i94cit_res.head()

code                                            country
0  582  MEXICO Air Sea, and Not Reported (I-94, no lan...
1  236                                        AFGHANISTAN
2  101                                            ALBANIA
3  316                                            ALGERIA
4  102                                            ANDORRA

In [10]:
i94port = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94prtl', ['code', 'port'])
i94port.head()

code                          port
0  ALC        ALCAN, AK             
1  ANC        ANCHORAGE, AK         
2  BAR  BAKER AAF - BAKER ISLAND, AK
3  DAC        DALTONS CACHE, AK     
4  PIZ    DEW STATION PT LAY DEW, AK

In [11]:
i94mode = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94model', ['code', 'mode'])
i94mode.head()

code          mode
0    1           Air
1    2           Sea
2    3          Land
3    9  Not reported

In [12]:
i94addr = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'i94addrl', ['code', 'addr'])
i94addr.head()

code        addr
0   AL     ALABAMA
1   AK      ALASKA
2   AZ     ARIZONA
3   AR    ARKANSAS
4   CA  CALIFORNIA

In [13]:
i94visa = sas_program_file_value_parser('data/I94_SAS_Labels_Descriptions.SAS', 'I94VISA', ['code', 'type'])
i94visa.head()

code      type
0    1  Business
1    2  Pleasure
2    3   Student

#### I94 Immigration Data prep

In [14]:
imm_data.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [15]:
imm_data.limit(10).toPandas()

cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode  \
0  5748517.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
1  5748518.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
2  5748519.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
3  5748520.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
4  5748521.0  2016.0     4.0   245.0   438.0     LOS  20574.0      1.0   
5  5748522.0  2016.0     4.0   245.0   464.0     HHW  20574.0      1.0   
6  5748523.0  2016.0     4.0   245.0   464.0     HHW  20574.0      1.0   
7  5748524.0  2016.0     4.0   245.0   464.0     HHW  20574.0      1.0   
8  5748525.0  2016.0     4.0   245.0   464.0     HOU  20574.0      1.0   
9  5748526.0  2016.0     4.0   245.0   464.0     LOS  20574.0      1.0   

  i94addr  depdate  ...  entdepu  matflag  biryear   dtaddto gender insnum  \
0      CA  20582.0  ...     None        M   1976.0  10292016      F   None   
1      NV  20591.0  ...     None        M   1984.0  10292016      F   None   
2      WA  20582.0  ...     None        M   1987.0  10292016      M   None   
3      WA  20588.0  ...     None        M   1987.0  10292016      F   None   
4      WA  20588.0  ...     None        M   1988.0  10292016      M   None   
5      HI  20579.0  ...     None        M   1959.0  10292016      M   None   
6      HI  20586.0  ...     None        M   1950.0  10292016      F   None   
7      HI  20586.0  ...     None        M   1975.0  10292016      F   None   
8      FL  20581.0  ...     None        M   1989.0  10292016      M   None   
9      CA  20581.0  ...     None        M   1990.0  10292016      F   None   

  airline        admnum  fltno visatype  
0      QF  9.495387e+10  00011       B1  
1      VA  9.495562e+10  00007       B1  
2      DL  9.495641e+10  00040       B1  
3      DL  9.495645e+10  00040       B1  
4      DL  9.495639e+10  00040       B1  
5      NZ  9.498180e+10  00010       B2  
6      NZ  9.497969e+10  00010       B2  
7      NZ  9.497975e+10  00010       B2  
8      NZ  9.497325e+10  00028       B2  
9      NZ  9.501355e+10  00002       B2  

[10 rows x 28 columns]

## Step 3: Define the Data Model <a name="step3"></a>

### 3.1 Conceptual Data Model <a name="data_model"></a>

Map out the conceptual data model and explain why you chose that model

The data model consists of tables `immigration`, `us_cities_demographics`, `airport_codes`, `world_temperature`, `i94cit_res`, `i94port`, `i94mode`, `i94addr`, `i94visa`

In addition to the schema below also note
1. In `immigration` table `i94mon` column is used as a DISTKEY AND `i94year` as SORTKEY
2. The following tables are distributed across all nodes(`DISTSTYLE ALL`): `us_cities_demographics`, `i94cit_res`, `i94port`, `i94mode`, `i94addr`, `i94visa`

<img src="imgs/data_model.png"/>

### 3.2 Mapping Out Data Pipelines <a name="pipeline_steps"></a>

Steps necessary to pipeline the data into the chosen data model:


    >> Begin Dummy Operator.

        >> Operator extract tables from I94 labels mappings files and stage to S3/local as csv:
            * i94cit_res
            * i94port
            * i94mode
            * i94addr
            * i94visa   
            >> Copy the above csv files from local/s3 to create tables in Redshift.
                >> Perform data quality checks for the tables above.
            >> Transform immigration data files on local/s3 and write results to `immigration` Redshift table.
                >> Perform data qualitiy checks for immigration table

        >> Copy csv files from local/s3 to create the following tables in Redshift.
            * us_cities_demographics
            * airport_codes
            * world_temperature
            >> Perform data quality checks on above tables.
            
                >> End Dummy Operator.
  

<img src="imgs/dag.png"/>

## Step 4: Run Pipelines to Model the Data <a name="step4"></a>

### 4.1 Create the data model

Build the data pipelines to create the data model.

Create Tables:
```bash
(venv) $ python create_tables.py
```

Launch Airflow UI:
1. Initialize Airflow & Run Webserver
```bash
(venv) $ export AIRFLOW_HOME=$(pwd)
(venv) $ airflow initdb
(venv) $ airflow webserver -p 8080
```
2. Run Scheduler (Open New Terminal Tab)
```bash
(venv) $ export AIRFLOW_HOME=$(pwd)
(venv) $ airflow scheduler
```
3. Access Airflow UI at `localhost:8080`
4. Run `etl_dag` in Airflow UI

## Step 5: Complete Project Write Up <a name="step5"></a>

### Technology Choices and tools

* Clearly state the rationale for the choice of tools and technologies for the project.

1. Apache Airflow: Allows for easy scheduling and monitoring etl workflows for keeping analytics database up to date
2. Redshift: For storing analytics tables in a distributed manner

### Data Schedule Proposal

* Propose how often the data should be updated and why.

Pipeline will be scheduled monthly as immigration data is the primary datasource is on a monthly granularity

### Possible Scenerios, changes and approach

* Write a description of how you would approach the problem differently under the following scenarios:-

 * **The data was increased by 100x:** Will have to use partitioning functionality in the dag, might also need to use Cloud services like AWS EMR to use spark for processing data.
 
 * **The data populates a dashboard that must be updated on a daily basis by 7am every day:** Will need to update the schedule of the DAG accordingly as make sure we have data needed for the dashboard.
 
 * **The database needed to be accessed by 100+ people:** Will create roles for the different people on AWS. That way different people have access to the relevant resources.